In [1]:
import math
import tensorflow as tf
import numpy as np
import matplotlib
import os
from tensorflow.contrib import rnn

tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)
##xy_with_noise = np.genfromtxt('/Users/yeseo/Desktop/2015eliminated_1.txt',delimiter = ',',dtype = None)

#data_preprocessing
xy= xy[:,:27]
a = xy[:,:2]
b = xy[:,2:]
b= MinMaxScaler(b)
xy = np.hstack((a,b))

##xy_with_noise = xy_with_noise[:,:27]
##a_with_noise = xy_with_noise[:,:2]
##b_with_noise = xy_with_noise[:,2:]
##b_with_noise = MinMaxScaler(b_with_noise)
##xy_with_noise = np.hstack((a_with_noise,b_with_noise))


#parameters
seq_length =6
data_dim =27
hidden_dim = 54
output_dim = 25
learning_rate = 0.01
iterations = 4000

train_size = int(len(xy)*0.7)
validation_size = int(len(xy)*0.2)

#divide data set to train,validation and test set
train_set = xy[:train_size]
validation_set = xy[train_size:train_size+validation_size]
test_set = xy[train_size+validation_size:]

##train_set_with_noise = xy_with_noise[:train_size]
##validation_set_with_noise = xy_with_noise[train_size:train_size+validation_size]
##test_set_with_noise = xy_with_noise[train_size+validation_size:]

# build data set for rnn
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
validationX, validationY = build_dataset(validation_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

##trainX_with_noise, trainY_with_noise = build_dataset(train_set_with_noise,seq_length)
##validationX_with_noise, validationY_with_noise = build_dataset(validation_set_with_noise,seq_length)
##testX_with_noise,testY_with_noise = build_dataset(test_set_with_noise, seq_length)


X1 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y1 = tf.placeholder(tf.float32,[None,25])
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

##X2 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
##Y2 = tf.placeholder(tf.float32,[None,25])

#LSTM CELL만들기

with tf.variable_scope("rnn1"):
    lstm_1 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    dropout1 = tf.contrib.rnn.DropoutWrapper(lstm_1,keep_prob)
    lstm_2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim*2, state_is_tuple = True, activation=tf.tanh)
    dropout2 = tf.contrib.rnn.DropoutWrapper(lstm_2,keep_prob)
    lstm_3 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    dropout3 = tf.contrib.rnn.DropoutWrapper(lstm_3,keep_prob)
    cell1 = tf.contrib.rnn.MultiRNNCell([dropout1,dropout2,dropout3])
    outputs1,_states1 = tf.nn.dynamic_rnn(cell1,X1,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs1[:,-1], output_dim,activation_fn = None)
    loss1 =tf.reduce_sum(tf.square(Y_pred-Y1))
    train1 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss1)

##with tf.variable_scope("rnn2"):
    ##cell2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    ##outputs2,_states2 = tf.nn.dynamic_rnn(cell2, X2, dtype = tf.float32)
    ##Y_pred_with_noise = tf.contrib.layers.fully_connected(outputs2[:,-1], output_dim,activation_fn = None)

    ##loss2 =tf.reduce_mean(tf.square(Y_pred_with_noise-Y2))
    ##train2 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss2)


#RMSE 측정
targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
x2 = x1+0.3
x3 = x2+0.3
loss_for_graph = np.zeros(iterations)
x4 = np.array(range(0,iterations))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss1 = sess.run([train1,loss1],feed_dict={X1:trainX, Y1:trainY, keep_prob : 0.7})
        print("[step: {}] loss: {}".format(i,step_loss1))
        loss_for_graph[i] = step_loss1
       ## _, step_loss2 = sess.run([train2,loss2],feed_dict={X2:trainX_with_noise, Y2:trainY_with_noise})
        ##print("[step: {}] loss: {}".format(i,step_loss2))
        
    validation_predict = sess.run(Y_pred, feed_dict = {X1:validationX, keep_prob : 1})
    ##validation_predict_with_noise = sess.run(Y_pred_with_noise, feed_dict = {X2:validationX_with_noise})
    test_predict = sess.run(Y_pred, feed_dict = {X1:testX, keep_prob : 1})
    
    rmse_val = sess.run(rmse, feed_dict={targets: validationY,predictions: validation_predict})
    print("RMSE: {}".format(rmse_val))
   # print("pred: {}".format(test_predict[-1,:]))
    #print("real: {}".format(testY[-1,:]))
    #print("noise: {}".format(eliminate_noise_pred[-1,:]))
    
#    plt.bar(x1,test_predict[-1,:],label = 'predict',color ='b',width = 0.1)
  #  plt.bar(x2,testY[-1,:],label = 'real',color ='g',width = 0.1)
    #plt.bar(x3,eliminate_noise_pred[-1,:],label = 'noise',color ='g',width = 0.1)
    plt.plot(x4,loss_for_graph)
    #plt.legend()
    plt.show()





Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
[step: 0] loss: 12119.41015625
[step: 1] loss: 4281.20751953125
[step: 2] loss: 9757.484375
[step: 3] loss: 3695.22119140625
[step: 4] loss: 3493.362060546875
[step: 5] loss: 3113.639892578125
[step: 6] loss: 3000.79248046875
[step: 7] loss: 2719.220703125
[step: 8] loss: 2279.71484375
[step: 9] loss: 1930.68115234375
[step: 10] loss: 1701.8218994140625
[step: 11] loss: 1603.5650634765625
[step: 12] loss: 1596.1556396484375
[step: 13] loss: 1596.169189453125
[step: 14] loss: 1578.91552734375
[step: 15] loss: 1582.922607421875
[step: 16] loss: 1538.53466796875
[step: 17] loss: 1492.2784423828125
[step: 18] loss: 1468.975341796875
[step: 19] loss: 1409.046142578125
[step: 20] loss: 1386.452392578125
[step: 21] loss: 1347.2236328125
[step: 22] loss: 1291.498535

[step: 219] loss: 218.46163940429688
[step: 220] loss: 218.74009704589844
[step: 221] loss: 213.45516967773438
[step: 222] loss: 217.34677124023438
[step: 223] loss: 217.61929321289062
[step: 224] loss: 217.01962280273438
[step: 225] loss: 221.06228637695312
[step: 226] loss: 218.4774627685547
[step: 227] loss: 214.9628448486328
[step: 228] loss: 205.9717559814453
[step: 229] loss: 211.05075073242188
[step: 230] loss: 212.60362243652344
[step: 231] loss: 214.89906311035156
[step: 232] loss: 212.88214111328125
[step: 233] loss: 215.60427856445312
[step: 234] loss: 209.1430206298828
[step: 235] loss: 211.0358123779297
[step: 236] loss: 225.60626220703125
[step: 237] loss: 212.98721313476562
[step: 238] loss: 222.890625
[step: 239] loss: 220.46649169921875
[step: 240] loss: 220.194580078125
[step: 241] loss: 210.15771484375
[step: 242] loss: 224.07955932617188
[step: 243] loss: 230.87112426757812
[step: 244] loss: 207.69674682617188
[step: 245] loss: 218.11302185058594
[step: 246] loss: 2

[step: 445] loss: 186.00729370117188
[step: 446] loss: 171.88204956054688
[step: 447] loss: 164.71072387695312
[step: 448] loss: 174.73890686035156
[step: 449] loss: 178.45523071289062
[step: 450] loss: 176.645263671875
[step: 451] loss: 172.35113525390625
[step: 452] loss: 179.71878051757812
[step: 453] loss: 174.23170471191406
[step: 454] loss: 174.3016357421875
[step: 455] loss: 178.15966796875
[step: 456] loss: 176.58731079101562
[step: 457] loss: 165.7836151123047
[step: 458] loss: 181.44570922851562
[step: 459] loss: 162.12637329101562
[step: 460] loss: 181.15750122070312
[step: 461] loss: 185.24517822265625
[step: 462] loss: 162.032470703125
[step: 463] loss: 178.06138610839844
[step: 464] loss: 177.85592651367188
[step: 465] loss: 169.42904663085938
[step: 466] loss: 181.15428161621094
[step: 467] loss: 162.47488403320312
[step: 468] loss: 175.10516357421875
[step: 469] loss: 166.7296600341797
[step: 470] loss: 167.18450927734375
[step: 471] loss: 160.91331481933594
[step: 472]

[step: 670] loss: 148.16114807128906
[step: 671] loss: 139.08627319335938
[step: 672] loss: 136.7813720703125
[step: 673] loss: 148.2603759765625
[step: 674] loss: 149.35894775390625
[step: 675] loss: 135.30740356445312
[step: 676] loss: 151.90089416503906
[step: 677] loss: 158.52259826660156
[step: 678] loss: 137.10235595703125
[step: 679] loss: 152.9486083984375
[step: 680] loss: 147.47203063964844
[step: 681] loss: 137.55914306640625
[step: 682] loss: 149.1881561279297
[step: 683] loss: 139.78451538085938
[step: 684] loss: 142.33807373046875
[step: 685] loss: 140.86782836914062
[step: 686] loss: 137.12515258789062
[step: 687] loss: 148.1187286376953
[step: 688] loss: 133.84140014648438
[step: 689] loss: 142.4013671875
[step: 690] loss: 139.99072265625
[step: 691] loss: 150.12615966796875
[step: 692] loss: 144.2705078125
[step: 693] loss: 134.6423797607422
[step: 694] loss: 150.47341918945312
[step: 695] loss: 134.00515747070312
[step: 696] loss: 143.53076171875
[step: 697] loss: 144

[step: 894] loss: 145.68641662597656
[step: 895] loss: 123.63702392578125
[step: 896] loss: 139.5668182373047
[step: 897] loss: 143.4307098388672
[step: 898] loss: 121.40093231201172
[step: 899] loss: 135.45913696289062
[step: 900] loss: 135.7253875732422
[step: 901] loss: 123.05416107177734
[step: 902] loss: 129.58822631835938
[step: 903] loss: 132.87440490722656
[step: 904] loss: 129.93704223632812
[step: 905] loss: 118.98963928222656
[step: 906] loss: 127.74329376220703
[step: 907] loss: 130.0518798828125
[step: 908] loss: 120.9385757446289
[step: 909] loss: 123.87305450439453
[step: 910] loss: 123.45474243164062
[step: 911] loss: 120.26295471191406
[step: 912] loss: 131.3386993408203
[step: 913] loss: 118.79006958007812
[step: 914] loss: 124.21763610839844
[step: 915] loss: 123.25433349609375
[step: 916] loss: 120.8037109375
[step: 917] loss: 122.35821533203125
[step: 918] loss: 123.36709594726562
[step: 919] loss: 115.87001037597656
[step: 920] loss: 116.69007873535156
[step: 921]

...

[step: 2892] loss: 71.49999237060547
[step: 2893] loss: 69.87519836425781
[step: 2894] loss: 68.74813842773438
[step: 2895] loss: 70.01251220703125
[step: 2896] loss: 66.62336730957031
[step: 2897] loss: 67.06632232666016
[step: 2898] loss: 69.06253051757812
[step: 2899] loss: 68.95654296875
[step: 2900] loss: 65.24885559082031
[step: 2901] loss: 71.04945373535156
[step: 2902] loss: 71.67021942138672
[step: 2903] loss: 72.22834777832031
[step: 2904] loss: 65.03582763671875
[step: 2905] loss: 65.27925109863281
[step: 2906] loss: 67.35226440429688
[step: 2907] loss: 68.14749145507812
[step: 2908] loss: 67.08529663085938
[step: 2909] loss: 66.4215087890625
[step: 2910] loss: 71.41728210449219
[step: 2911] loss: 74.60556030273438
[step: 2912] loss: 70.24964904785156
[step: 2913] loss: 65.53846740722656
[step: 2914] loss: 66.87637329101562
[step: 2915] loss: 70.90021514892578
[step: 2916] loss: 68.12799072265625
[step: 2917] loss: 68.46630859375
[step: 2918] loss: 62.93479537963867
[step: 2

[step: 3115] loss: 64.34233093261719
[step: 3116] loss: 75.65908813476562
[step: 3117] loss: 74.49488830566406
[step: 3118] loss: 65.8626708984375
[step: 3119] loss: 68.79972839355469
[step: 3120] loss: 80.02474975585938
[step: 3121] loss: 69.42919921875
[step: 3122] loss: 66.03258514404297
[step: 3123] loss: 74.93244934082031
[step: 3124] loss: 61.283935546875
[step: 3125] loss: 69.60735321044922
[step: 3126] loss: 64.81637573242188
[step: 3127] loss: 66.87118530273438
[step: 3128] loss: 67.17525482177734
[step: 3129] loss: 68.06747436523438
[step: 3130] loss: 63.93009948730469
[step: 3131] loss: 64.17430877685547
[step: 3132] loss: 65.08183288574219
[step: 3133] loss: 66.47816467285156
[step: 3134] loss: 67.7530288696289
[step: 3135] loss: 62.301300048828125
[step: 3136] loss: 61.24790954589844
[step: 3137] loss: 65.10920715332031
[step: 3138] loss: 62.88028335571289
[step: 3139] loss: 65.64373779296875
[step: 3140] loss: 65.27367401123047
[step: 3141] loss: 63.26871871948242
[step: 

[step: 3337] loss: 69.97055053710938
[step: 3338] loss: 65.10556030273438
[step: 3339] loss: 73.065185546875
[step: 3340] loss: 72.54435729980469
[step: 3341] loss: 60.62964630126953
[step: 3342] loss: 74.40435791015625
[step: 3343] loss: 70.55754089355469
[step: 3344] loss: 63.755393981933594
[step: 3345] loss: 72.13233947753906
[step: 3346] loss: 69.48735046386719
[step: 3347] loss: 62.218162536621094
[step: 3348] loss: 62.13855743408203
[step: 3349] loss: 63.576148986816406
[step: 3350] loss: 63.14203643798828
[step: 3351] loss: 68.01477813720703
[step: 3352] loss: 63.65373611450195
[step: 3353] loss: 62.068359375
[step: 3354] loss: 62.62921905517578
[step: 3355] loss: 68.90426635742188
[step: 3356] loss: 67.92849731445312
[step: 3357] loss: 62.88325119018555
[step: 3358] loss: 67.80168151855469
[step: 3359] loss: 70.52336120605469
[step: 3360] loss: 62.06474304199219
[step: 3361] loss: 67.71249389648438
[step: 3362] loss: 67.71908569335938
[step: 3363] loss: 63.79596710205078
[step

[step: 3559] loss: 62.81969451904297
[step: 3560] loss: 64.78141021728516
[step: 3561] loss: 63.433353424072266
[step: 3562] loss: 62.18376922607422
[step: 3563] loss: 60.692283630371094
[step: 3564] loss: 61.874717712402344
[step: 3565] loss: 63.305477142333984
[step: 3566] loss: 62.46942138671875
[step: 3567] loss: 58.86699676513672
[step: 3568] loss: 60.43608093261719
[step: 3569] loss: 60.80526351928711
[step: 3570] loss: 58.93825912475586
[step: 3571] loss: 62.34556579589844
[step: 3572] loss: 62.88862609863281
[step: 3573] loss: 64.0010986328125
[step: 3574] loss: 72.27804565429688
[step: 3575] loss: 66.13421630859375
[step: 3576] loss: 59.598873138427734
[step: 3577] loss: 62.92044448852539
[step: 3578] loss: 77.07185363769531
[step: 3579] loss: 90.93097686767578
[step: 3580] loss: 66.45863342285156
[step: 3581] loss: 77.08177185058594
[step: 3582] loss: 83.68306732177734
[step: 3583] loss: 63.8825569152832
[step: 3584] loss: 78.66728210449219
[step: 3585] loss: 77.9488601684570

[step: 3781] loss: 63.96527099609375
[step: 3782] loss: 63.8216552734375
[step: 3783] loss: 58.656463623046875
[step: 3784] loss: 56.242210388183594
[step: 3785] loss: 58.225650787353516
[step: 3786] loss: 59.31947326660156
[step: 3787] loss: 61.51908493041992
[step: 3788] loss: 60.226470947265625
[step: 3789] loss: 59.762298583984375
[step: 3790] loss: 61.0732421875
[step: 3791] loss: 64.62940979003906
[step: 3792] loss: 74.37984466552734
[step: 3793] loss: 67.07571411132812
[step: 3794] loss: 58.53966522216797
[step: 3795] loss: 59.34843444824219
[step: 3796] loss: 62.43701171875
[step: 3797] loss: 60.51819610595703
[step: 3798] loss: 60.66337585449219
[step: 3799] loss: 58.59236145019531
[step: 3800] loss: 58.59233093261719
[step: 3801] loss: 63.36058807373047
[step: 3802] loss: 58.817726135253906
[step: 3803] loss: 57.93736267089844
[step: 3804] loss: 61.516029357910156
[step: 3805] loss: 61.58709716796875
[step: 3806] loss: 58.513580322265625
[step: 3807] loss: 58.850494384765625
